In [71]:
#Importing Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import pickle
import warnings
import re
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [72]:
df1=pd.read_csv("movies.csv")
print("Dimension of Dataframe is : ",df1.shape)
df1.head()

Dimension of Dataframe is :  (10329, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [73]:
df1['title'] = df1['title'].str.extract('(.+?) \(')
df1.head()

,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [74]:
df1[df1['movieId']==7]

,movieId,title,genres
6,7,Sabrina,Comedy|Romance


In [75]:
print(df1['title'][10])

American President, The


In [76]:
preprocessed_Phrase = []
# tqdm is for printing the status bar
for sentance in tqdm(df1['title'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() )
    preprocessed_Phrase.append(sentance.strip())

100%|█████████████████████████████████████████████████████████████████████████| 10329/10329 [00:00<00:00, 47812.62it/s]


In [77]:
df1['title']=preprocessed_Phrase

In [78]:
print(df1['title'][10])

american president the


In [79]:
preprocessed = []
# tqdm is for printing the status bar
for sentance in tqdm(df1['genres'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() )
    preprocessed.append(sentance.strip())

100%|█████████████████████████████████████████████████████████████████████████| 10329/10329 [00:00<00:00, 37695.06it/s]


In [80]:
df1['genres']=preprocessed

In [81]:
df1.head()

,movieId,title,genres
0,1,toy story,adventure animation children comedy fantasy
1,2,jumanji,adventure children fantasy
2,3,grumpier old men,comedy romance
3,4,waiting to exhale,comedy drama romance
4,5,father of the bride part ii,comedy


In [82]:
df1=df1.iloc[:-4]

In [83]:
# Total number of different movies
print("Total Different Movies are :  ",len(df1['title'].unique()))

Total Different Movies are :   9742


In [84]:
df1['genres'].value_counts() # More genre is drama

drama                                          1384
comedy                                          824
comedy drama                                    465
drama romance                                   421
comedy romance                                  363
                                               ... 
adventure drama film noir thriller                1
adventure animation children fantasy sci fi       1
adventure thriller western                        1
comedy drama romance sci fi                       1
adventure animation children drama musical        1
Name: genres, Length: 938, dtype: int64

In [85]:
df1.loc[0:5,'genres']

0    adventure animation children comedy fantasy
1                     adventure children fantasy
2                                 comedy romance
3                           comedy drama romance
4                                         comedy
5                          action crime thriller
Name: genres, dtype: object

In [86]:
tfidf=TfidfVectorizer().fit_transform(df1['genres'])
tfidf.shape

(10325, 24)

In [87]:
matrix_1=pd.DataFrame(tfidf.toarray(),index=df1['title'].tolist())

In [88]:
matrix_1.head(20)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
toy story,0.000000,0.404089,0.539215,0.501501,0.263883,0.000000,0.0,0.000000,0.474153,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
jumanji,0.000000,0.505264,0.000000,0.627066,0.000000,0.000000,0.0,0.000000,0.592871,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
grumpier old men,0.000000,0.000000,0.000000,0.000000,0.602438,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.798166,0.0,0.000000,0.0,0.0
waiting to exhale,0.000000,0.000000,0.000000,0.000000,0.541474,0.000000,0.0,0.438349,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.717395,0.0,0.000000,0.0,0.0
father of the bride part ii,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
heat,0.579228,0.000000,0.000000,0.000000,0.000000,0.618246,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.531287,0.0,0.0
sabrina,0.000000,0.000000,0.000000,0.000000,0.602438,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.798166,0.0,0.000000,0.0,0.0
tom and huck,0.000000,0.627426,0.000000,0.778676,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
sudden death,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
goldeneye,0.563474,0.644497,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.516837,0.0,0.0


In [89]:
# saving dataframe to csv for later use in main file
df1.to_csv('data.csv',index=False)

# Creating Collaboration Filtering Using Rating dataset

In [14]:
#Reading Ratings.csv File
df2=pd.read_csv("ratings.csv")
print("Dimension of Rating Dataframe :",df2.shape)
df2.head()



Dimension of Rating Dataframe : (105339, 4)


,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [15]:
# Total number of different movies
print("Total Different Movies are :  ",len(df2['movieId'].unique()))

Total Different Movies are :   10325


In [16]:
#Mergeing Two Dataframes
rating=pd.merge(df1[['movieId']],df2,on='movieId',how='right')

In [17]:
# Creating table that contains only movieId,userId and ratings
rating_f=rating.pivot(index='movieId',columns='userId',values='rating').fillna(0)

In [18]:
rating_f.head()

userId,1,2,3,4,5,6,7,8,9,10,...,659,660,661,662,663,664,665,666,667,668
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,5.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0
3,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5


In [19]:
rating_f.shape

(10325, 668)

# Since ot is too large So we will use SVD to Decompose it

In [20]:
svd=TruncatedSVD(n_components=200).fit_transform(rating_f)
matrix_2=pd.DataFrame(svd,index=df1['title'].tolist())
matrix_2.shape

(10325, 200)

In [21]:
matrix_2.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
toy story (1995),39.537219,-8.970713,10.000227,0.731054,4.866800,9.104943,2.767281,3.932202,-4.136125,-4.210637,...,0.183642,-2.044815,-1.287747,3.213063,1.346297,0.040482,-0.606629,0.944513,-0.150359,-0.873046
jumanji (1995),17.361962,-1.567811,4.453533,6.037296,0.477950,1.383908,4.717559,1.832270,2.687148,4.551667,...,0.027040,1.172631,0.409541,0.515612,-1.049877,0.496331,-0.725381,-0.987865,-0.957144,-1.117020
grumpier old men (1995),7.128164,-2.930619,-0.503140,5.679298,-3.217641,0.292688,1.026978,2.252096,1.345689,-4.060691,...,-0.317012,0.717011,0.068709,0.704266,-0.174236,0.763384,0.551485,0.077931,-0.162762,-0.024026
waiting to exhale (1995),1.940867,-2.005853,-0.856432,1.288147,-0.163616,-1.361831,-0.651205,1.067695,-0.384301,-0.765436,...,0.237083,-0.132604,0.537012,-0.158076,0.138338,0.300832,0.733257,-0.002620,0.123151,0.056696
father of the bride part ii (1995),7.420355,-2.950298,-0.722306,5.628852,-1.242630,2.614648,-1.083076,1.764885,-0.650604,-3.917615,...,1.221959,0.127861,-0.177053,0.078484,0.555586,0.345030,0.861522,1.054496,-0.346464,-1.105485


# Creating and Running ContentBased Filtering using K-NearestNeighbors

In [22]:
matrix_3=pd.DataFrame(rating_f,index=df1['title'].tolist()).fillna(0)
matrix_3.shape

(10325, 668)

In [23]:
# Since this data contains many value zero.So we will put it into sparse matrix
movie_features_df_matrix = csr_matrix(matrix_3.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [24]:
print("Shape after storing in Sparse matrix : ",movie_features_df_matrix .shape)

Shape after storing in Sparse matrix :  (10325, 668)


In [25]:
query_index = np.random.choice(matrix_3.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(matrix_3.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

4976


In [26]:

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(matrix_3.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, matrix_3.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for come and see (idi i smotri) (1985):

1: lies my father told me (1975), with distance of 1.0:
2: 49th parallel (1941), with distance of 1.0:
3: lady vengeance (sympathy for lady vengeance) (chinjeolhan geumjassi) (2005), with distance of 1.0:
4: district 13 (banlieue 13) (2004), with distance of 1.0:
5: voices of a distant star (hoshi no koe) (2003), with distance of 1.0:


# Running Content / Collaborative and Cosine similarity

In [27]:
a_1=np.array(matrix_1.loc['toy story (1995)']).reshape(1,-1)
a_2=np.array(matrix_2.loc['toy story (1995)']).reshape(1,-1)
# Calculate the similarity of this two movie with others

score_1=cosine_similarity(matrix_1,a_1).reshape(-1)
score_2=cosine_similarity(matrix_2,a_2).reshape(-1)

#Average Measure of both content based and collaborated based filtering
hybrid=((score_1 + score_2)/2.0)

#Form a dataframe for similar movies
Df=pd.DataFrame({'Content': score_1,'Collaborative':score_2,'Hybrid':hybrid},index=matrix_2.index)
# Sort on the basis of Either Content,collaborative or hybrid
# Here Content
Df.sort_values('Content',ascending=False,inplace=True)
Df[1:].head(15)


,Content,Collaborative,Hybrid
"adventures of rocky and bullwinkle, the (2000)",1.000000,0.152299,0.576150
shrek the third (2007),1.000000,0.262796,0.631398
"wild, the (2006)",1.000000,0.032332,0.516166
ducktales: the movie - treasure of the lost lamp (1990),1.000000,0.355086,0.677543
asterix and the vikings (astérix et les vikings) (2006),1.000000,0.055286,0.527643
"monsters, inc. (2001)",1.000000,0.519742,0.759871
"tale of despereaux, the (2008)",1.000000,0.041607,0.520804
"emperor's new groove, the (2000)",1.000000,0.335003,0.667501
turbo (2013),1.000000,0.093143,0.546571
"boxtrolls, the (2014)",1.000000,0.141760,0.570880


In [28]:
sim_1=cosine_similarity(matrix_1)
sim_2=cosine_similarity(matrix_2)


In [29]:
# Saving into pickle
pickle.dump(sim_1,open('sim_1.pkl','wb'))
pickle.dump(sim_2,open('sim_2.pkl','wb'))